In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [2]:
df = pd.read_csv('data_for_model.csv', usecols = ['review_rating' , 'Reviews_tokenize_join'])

In [3]:
df['review_rating'].value_counts()

5    11878
4     6622
3     3074
1     1741
2     1735
Name: review_rating, dtype: int64

In [5]:
#df.info()
df['target'] = df['review_rating'].apply(lambda x: str(x))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25050 entries, 0 to 25049
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   review_rating          25050 non-null  int64 
 1   Reviews_tokenize_join  25050 non-null  object
 2   target                 25050 non-null  object
dtypes: int64(1), object(2)
memory usage: 587.2+ KB


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['Reviews_tokenize_join'] ,df['target'], test_size=0.2)


<IPython.core.display.Javascript object>

In [10]:

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [11]:
model = pipeline.fit(X_train, y_train)


In [12]:
print("accuracy score: " + str(model.score(X_test, y_test)))


accuracy score: 0.6215568862275449


In [13]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']


In [17]:

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)
feature_names[:10]

<IPython.core.display.Javascript object>

array(['aand price', 'able coffee', 'able comfortable', 'able complain',
       'able cook', 'able manually', 'able rebook', 'able suitcase',
       'able text', 'able toilet'], dtype='<U28')

In [15]:
target_names = ['1', '2', '3', '4', '5']
print("top 10 keywords per class:")
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print("%s: %s" % (label, " ".join(feature_names[top10])))

top 10 keywords per class:


<IPython.core.display.Javascript object>

1: total cost horrible ruin uncleaned experience superb avoid cost stay away disgusting bed uncomfortable bad


<IPython.core.display.Javascript object>

2: work leak sink didn book site like hadn rudest shoddy wasn worth soil feel dirty maze


...

3: room rude average average experience worth lobby date staff tall building pro despite request problem need leave floor


<IPython.core.display.Javascript object>

4: attend opera lobby large good reasonable overall location downside reason didn simple clean excellent right general good drawback


<IPython.core.display.Javascript object>

5: perfect love worth penny andrew grateful absolutely stay emanuel exceed impressed overall excellent


In [ ]:
## limpiar la data quetar names como did